# Entity Matching

The process of finding matching records in tables is called entity matching. It can be used for deduplication of tables or to link tables to merge the information that they contain. The question that we want to answer in this notebook is whether machine learning has something to add to traditional programming methods for entity matching and whether fancy deep learning approaches really outperform simpler machine learning moedels that have already been around for decades.

To investigate this, we make use of two existing entity matching packages: the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/) (PRLT), which offers support for traditional programmimng techniques and basic machine learning models and [DeepMatcher](https://anhaidgroup.github.io/deepmatcher/html/), a package that adopts deep learning techniques. We test both packages on  scientific bibliographic data from [ACM](https://www.acm.org/) and [DBLP](https://dblp.org/), which we downloaded from the [Database Group Leipzig ](https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution). This is an independent dataset that was not used to advertise either of the two methods.

## Package Imports

We now import the packages that we are going to use, together with our self-created module em_helper that contains some utitility functions to generate and display clasification performance metrics.

In [1]:
import recordlinkage
import pandas as pd
import numpy as np
import deepmatcher as dm
from em_helper import print_results, performance_metrics

## Data Loading
Having all our dependencies in place, we now load and display the two tables that contain the records that we want to match.

In [2]:
left_df = pd.read_csv('Data/DBLP2.csv', encoding='cp1252')
right_df = pd.read_csv('Data/ACM.csv', encoding='cp1252')
left_df.set_index('id',inplace=True)
right_df.set_index('id',inplace=True)
display(left_df)
display(right_df)

,title,authors,venue,year
id,,,,
journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995
...,...,...,...,...
journals/tods/KarpSP03,A simple algorithm for finding frequent elemen...,"Scott Shenker, Christos H. Papadimitriou, Rich...",ACM Trans. Database Syst.,2003
conf/vldb/LimWV03,SASH: A Self-Adaptive Histogram Set for Dynami...,"Lipyeow Lim, Min Wang, Jeffrey Scott Vitter",VLDB,2003
journals/tods/ChakrabartiKMP02,Locally adaptive dimensionality reduction for ...,"Kaushik Chakrabarti, Eamonn J. Keogh, Michael ...",ACM Trans. Database Syst.,2002


,title,authors,venue,year
id,,,,
304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999
...,...,...,...,...
672977,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",Very Large Data Bases,1994
950482,Guest editorial,"Philip A. Bernstein, Yannis Ioannidis, Raghu R...",The VLDB Journal &mdash; The International Jou...,2003
672980,GraphDB: Modeling and Querying Graphs in Datab...,Ralf Hartmut G&#252;ting,Very Large Data Bases,1994


# Defining Candidate Links
To translate the matching problem into a standard classification problem, we first form all possible combinations of rows in the DBLP table and rows in the ACM table. In other words, we form the Cartesian product of the indices of these tables. The DBLP table has 2616 rows and the the ACM table has 2294 rows, so this results in a collection of 2616 x 2294 = 6001104 combinations. 

The size of this set of possible links now places us for a dilemma:
1. We would like to use as much of the data as possible to reach a sound conclusion
2. DeepMatcher is not that fast and will take ages to process such a  large dataset.

To solve this dilemma, we apply a technique that is commonly used in entity matching: _blocking_. By using a simple heuristic, we filter out row combinations, of which we can be (almost) certain that they do not match. By blocking these row combinations, we end up with a considerably smaller set of candidate links that is much easier to manage. 

To perform blocking, we make use of the string comparison capabilities of the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/). We block all combinations with a [Lehvenstein-distance](https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance)-based similarity measure that is smaller than 0.3, where 0 indicates a complete mismatch and 1 indicates identical strings. By doing this, we reduce the number of candidate links to 162894, which amounts to a data reduction of about 97%.

In [3]:
# Full link
indexer = recordlinkage.Index()
indexer.full()
links = indexer.index(left_df, right_df)
display(links)

# Comparison step for blocking
compare_cl = recordlinkage.Compare()
compare_cl.string('title', 'title', threshold=0.3, label='title')
features = compare_cl.compute(links, left_df, right_df)

# Blocking
candidate_links = links[features.sum(axis=1) > 0]
display(candidate_links)

MultiIndex([('journals/sigmod/Mackay99', 304586),
            ('journals/sigmod/Mackay99', 304587),
            ('journals/sigmod/Mackay99', 304589),
            ('journals/sigmod/Mackay99', 304590),
            ('journals/sigmod/Mackay99', 304582),
            ('journals/sigmod/Mackay99', 304583),
            ('journals/sigmod/Mackay99', 304584),
            ('journals/sigmod/Mackay99', 304585),
            ('journals/sigmod/Mackay99', 306112),
            ('journals/sigmod/Mackay99', 306115),
            ...
            (         'conf/vldb/LiM01', 945735),
            (         'conf/vldb/LiM01', 672983),
            (         'conf/vldb/LiM01', 950484),
            (         'conf/vldb/LiM01', 672978),
            (         'conf/vldb/LiM01', 950483),
            (         'conf/vldb/LiM01', 672977),
            (         'conf/vldb/LiM01', 950482),
            (         'conf/vldb/LiM01', 672980),
            (         'conf/vldb/LiM01', 945741),
            (         'conf/vldb/L

MultiIndex([('journals/sigmod/Mackay99', 304570),
            ('journals/sigmod/Mackay99', 309852),
            ('journals/sigmod/Mackay99', 309849),
            ('journals/sigmod/Mackay99', 309895),
            ('journals/sigmod/Mackay99', 310061),
            ('journals/sigmod/Mackay99', 309897),
            ('journals/sigmod/Mackay99', 276345),
            ('journals/sigmod/Mackay99', 304183),
            ('journals/sigmod/Mackay99', 375733),
            ('journals/sigmod/Mackay99', 375769),
            ...
            (         'conf/vldb/LiM01', 872796),
            (         'conf/vldb/LiM01', 872809),
            (         'conf/vldb/LiM01', 872815),
            (         'conf/vldb/LiM01', 767135),
            (         'conf/vldb/LiM01', 603868),
            (         'conf/vldb/LiM01', 958947),
            (         'conf/vldb/LiM01', 672818),
            (         'conf/vldb/LiM01', 959076),
            (         'conf/vldb/LiM01', 672827),
            (         'conf/vldb/L

## Inspection of blocking effects
Since blocking might have the unwanted effect of throwing away a considerable number of true mtaches, we briefly verify whether this is the case. We load the data set containing the true links and see how much of these links are still present in the blocked data set. It turns out that only 8 of the 2224 true matches are discarded by the blocking procesdure, which amounts to 0.36%. This seems to be a defendable sacrifice to make for the 97% data reduction that is achieved by blocking.

In [21]:
matches = pd.read_csv('Data/DBLP-ACM_perfectMapping.csv', encoding='cp1252')
matches_tuples = list(matches.itertuples(index=False, name=None)) 
candidate_links_tuples = candidate_links.tolist()
intersection = set(candidate_links_tuples).intersection(matches_tuples)
display(len(intersection))
display(len(matches_tuples))

2216

2224

## Data preparation for DeepMatcher
To use the DeepMatcher classifier, we need to transform our data into something that [DeepMatcher](https://anhaidgroup.github.io/deepmatcher/html/) can understand. DeepMatcher takes in a table with record pairs of the two tables, which need to be matched. The records of the first table  are marked by adding the prefix "left_" to their feature names, and the records of the second table are marked by adding the prefix "right_". The target variable that indicates whether a pair forms a match is called "label" and takes on the vlaues 0 (no match) or 1 (match). To prevent unwanted correlations between the subsequent rows, we additionally shuffle the table.

In [23]:
left_df_renamed =left_df.reset_index().add_prefix('left_')
left_df_renamed['join'] = 1
right_df_renamed =right_df.reset_index().add_prefix('right_')
right_df_renamed['join'] = 1
final_df_full = pd.merge(left_df_renamed, right_df_renamed, on="join")
final_df_full.insert(0,'label',0)
final_df_full['combined_index'] = list(zip(final_df_full.left_id, final_df_full.right_id))
final_df_full.loc[final_df_full['combined_index'].isin(matches_tuples),'label'] = 1
final_df_blocked = final_df_full.loc[final_df_full['combined_index'].isin(candidate_links)]
final_df = final_df_blocked.drop(columns=['combined_index','left_id','right_id','join'])
final_df.index.name = 'id'
final_df = final_df.sample(frac=1)
display(final_df)

,label,left_title,left_authors,left_venue,left_year,right_title,right_authors,right_venue,right_year
id,,,,,,,,,
4003186,0,Semistructured und Structured Data in the Web:...,"Paolo Merialdo, Giansalvatore Mecca, Paolo Atzeni",SIGMOD Record,1997,Olympic records for data at the 1998 Nagano games,Edwin R. Lassettre,International Conference on Management of Data,1998
485817,0,An Automated System for Web Portal Personaliza...,"Philip S. Yu, Charu C. Aggarwal",VLDB,2002,Automatic subspace clustering of high dimensio...,"Rakesh Agrawal, Johannes Gehrke, Dimitrios Gun...",International Conference on Management of Data,1998
2581657,0,Declustering of Key-Based Partitioned Signatur...,"Pavel Zezula, Paolo Ciaccia, Paolo Tiberio",ACM Trans. Database Syst.,1996,Distinguished database profiles,Marianne Winslett,ACM SIGMOD Record,2002
3835435,0,The Jungle Database Search Engine,"Linas Bukauskas, Curtis E. Dyreson, Michael H....",SIGMOD Conference,1999,"Very Large Databases: How Large, How Different?",David Vaskevitch,Very Large Data Bases,1995
2575416,0,Information integration on the Web: a view fro...,"Subbarao Kambhampati, Craig A. Knoblock",SIGMOD Record,2003,Data extraction and transformation for the dat...,Case Squire,International Conference on Management of Data,1995
...,...,...,...,...,...,...,...,...,...
5959276,0,An Open Storage System for Abstract Objects,"Lukas Relly, Stephen Blott, Hans-Jörg Schek",SIGMOD Conference,1996,Lore: a database management system for semistr...,"Jason McHugh, Serge Abiteboul, Roy Goldman, Da...",ACM SIGMOD Record,1997
5809458,0,Optimizing Queries on Compressed Bitmaps,"Sihem Amer-Yahia, Theodore Johnson",VLDB,2000,Reordering Query Execution in Tertiary Memory ...,"Sunita Sarawagi, Michael Stonebraker",Very Large Data Bases,1996
250584,0,ARIADNE: A System for Constructing Mediators f...,"Ion Muslea, Greg Barish, Andrew Philpot, Craig...",SIGMOD Conference,1998,Cost models for overlapping and multiversion s...,"Yufei Tao, Dimitris Papadias, Jun Zhang",ACM Transactions on Database Systems (TODS),2002


## Creating train/validation/test sets
Now that our dataset is the right format, we create a train, validation and test set by applying a 60:20:20 split. The resulting datasets are saved to csv-files inside the Data directory.

In [24]:
train_split,validate_split,test_split = np.split(final_df, [int(.6 * len(final_df)), int(.8 * len(final_df))])
train_file = 'Data/train_block.csv'
validate_file = 'Data/validate_block.csv'
test_file = 'Data/test_block.csv'
train_split.to_csv(train_file)
validate_split.to_csv(validate_file)
test_split.to_csv(test_file)

## Data preprocessing and feature engineering
We now let [DeepMatcher](https://anhaidgroup.github.io/deepmatcher/html/) apply its standard data preprecessing and feature engineering procedure. Due to the time limits of the Project Challenge, we have not delved deeper into this, but rather used it as a black box. To get a rough idea of what [DeepMatcher](https://anhaidgroup.github.io/deepmatcher/html/) does behind the scenes, we display the still human-interpretable raw data table, which holds the result of text normalization.

In [53]:
train, validate, test = dm.data.process(
    path='.',
    left_prefix='left_',
    right_prefix='right_',
    label_attr='label',
    id_attr='id',
    cache=None,
    train=train_file,
    validation=validate_file,
    test=test_file)
train_table = train.get_raw_table()
display(train_table)


Reading and processing data from ".\Data/train_block.csv"
0% [##############################] 100% | ETA: 00:00:00
Reading and processing data from ".\Data/validate_block.csv"
0% [##############################] 100% | ETA: 00:00:00
Reading and processing data from ".\Data/test_block.csv"
0% [##############################] 100% | ETA: 00:00:00
Building vocabulary
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05

Computing principal components
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:09


,id,label,left_title,left_authors,left_venue,left_year,right_title,right_authors,right_venue,right_year
0,4003186,0,semistructured und structured data in the web ...,"paolo merialdo , giansalvatore mecca , paolo a...",sigmod record,1997,olympic records for data at the 1998 nagano games,edwin r. lassettre,international conference on management of data,1998
1,485817,0,an automated system for web portal personaliza...,"philip s. yu , charu c. aggarwal",vldb,2002,automatic subspace clustering of high dimensio...,"rakesh agrawal , johannes gehrke , dimitrios g...",international conference on management of data,1998
2,2581657,0,declustering of key-based partitioned signatur...,"pavel zezula , paolo ciaccia , paolo tiberio",acm trans . database syst .,1996,distinguished database profiles,marianne winslett,acm sigmod record,2002
3,3835435,0,the jungle database search engine,"linas bukauskas , curtis e. dyreson , michael ...",sigmod conference,1999,"very large databases : how large , how differe...",david vaskevitch,very large data bases,1995
4,2575416,0,information integration on the web : a view fr...,"subbarao kambhampati , craig a. knoblock",sigmod record,2003,data extraction and transformation for the dat...,case squire,international conference on management of data,1995
...,...,...,...,...,...,...,...,...,...,...
97731,4554620,0,static detection of security flaws in object-o...,keishi tajima,sigmod conference,1996,integrating reliable memory in databases,"wee teck ng , peter m. chen",very large data bases,1997
97732,4959874,0,foundations of preferences in database systems,werner kießling,vldb,2002,metu interoperable database system,"asuman dogac , ugur halici , ebru kilic , gokh...",international conference on management of data,1996
97733,3486577,0,a data warehousing architecture for enabling s...,yannis kotidis,vldb,2001,query execution techniques for caching expensi...,"joseph m. hellerstein , jeffrey f. naughton",international conference on management of data,1996
97734,1168443,0,aqr-toolkit : an adaptive query routing middle...,"henrique paques , wei tang , calton pu , wei h...",sigmod conference,2000,an adaptive hybrid server architecture for cli...,"kaladhar voruganti , m. tamer & # 214 ; zsu , ...",very large data bases,1999


## DeepMatcher Model Training
We are now ready to train our DeepMatcher model. We take over the values of he DeepMatcher ["Getting started"-notebook](https://github.com/anhaidgroup/deepmatcher/blob/master/examples/getting_started.ipynb), but make a couple of small adjustments. Knowing that an epoch will take about half an hour on Google Colab and 2 hours on our local machine, we limit the number of epochs to 8 instead of 10, and since we have not seen any substantial benefit of weighting in smaller experiments, we practically disable it by setting pos_neg_ratio = 1.

The training function prints the performance metric F1-measure, precision and recall for both the training and the validation set at each epoch and uses the F1-measure on the validation set to select the best model. The F1-measure is more suitable than accuracy in thie case, because even in the blocked dataset, the fraction of true matches is very small (about 1.4% ).

In [26]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.run_train(
    train,
    validate,
    epochs=8,
    batch_size=16,
    best_save_path='Results/model_dblp_acm_block.pth',
    pos_neg_ratio=1)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


c:\python-3.6.3\lib\site-packages\torch\nn\functional.py:1946: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 01:12:35


Finished Epoch 1 || Run Time: 3032.0 | Load Time: 1326.8 || F1:  92.83 | Prec:  95.97 | Rec:  89.90 || Ex/s:  22.42

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:14:04


Finished Epoch 1 || Run Time:  385.9 | Load Time:  459.1 || F1:  97.75 | Prec:  98.86 | Rec:  96.66 || Ex/s:  38.55

* Best F1: tensor(97.7477, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 01:12:08


Finished Epoch 8 || Run Time: 3014.6 | Load Time: 1316.6 || F1:  99.69 | Prec:  99.85 | Rec:  99.54 || Ex/s:  22.57

===>  EVAL Epoch 8


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:13:41


Finished Epoch 8 || Run Time:  375.5 | Load Time:  446.3 || F1:  98.22 | Prec:  98.22 | Rec:  98.22 || Ex/s:  39.64

---------------------

Loading best model...
Training done.


tensor(98.4340, device='cuda:0')

## DeepMatcher Testing
We are now curious to see what the selected model does on the test set. We see that compared to the training and validation results, there is no serious drop in the F1-value, so we are reasonably confident that our model is not overfitting.

In [28]:
model.run_eval(test)

===>  EVAL Epoch 5


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:11:52


Finished Epoch 5 || Run Time:  209.1 | Load Time:  506.0 || F1:  97.98 | Prec:  97.88 | Rec:  98.09 || Ex/s:  45.56



tensor(97.9809, device='cuda:0')

##  PRLT Feature Engineering
We are now ready to switch to the the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/), which uses a simpler approach to feature engineering. Basically, we fill a vector with zeros of ones that indicate whether a record pair matches with respect to a certain column. So for each of the columns in the left/right tables, we define how similar they need to be to be matching. All the values of the per-column-matches are then concatenated to a global feature vector of zeros and ones, where  an all zero vector means that the pair matches in none of the columns, an an all-one-vectir indicates a match with respect to every column. This vector of zeros and ones is then fed to the various classifier models.

To define when we call two colum values matching, we let ourselves be inspired by the [example](https://recordlinkage.readthedocs.io/en/latest/notebooks/link_two_dataframes.html) that the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/) provides in its introductory text. Here most of the column matches are defined in terms of a similarity that is based on the [Jaro Winkler distance](https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance). The only small adaptation that we make is in the choice of the similarity threshold value for "venue", because a short inspection of the data indicates that matching records tend to agree less with respect to "venue" than they do with respect to "title" or 'author".

In [32]:
# Column-based matching criteria that define the feature vector
compare_cl = recordlinkage.Compare()
compare_cl.string('title', 'title', method='jarowinkler', threshold=0.85,label = 'title')
compare_cl.string('authors', 'authors', method='jarowinkler', threshold=0.85,label = 'authors')
compare_cl.exact('year', 'year', label='year')
compare_cl.string('venue', 'venue', method='jarowinkler', threshold=0.65,label = 'authors')

<Compare>

## PRLT Data Loading and Feature Computation
We now load our previously saved train/test/validation sets from file. Ironically, to feed the data to the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/) methods, we need to split up the joined table into two separate tables and provide a list of links indicating which record combinations need to be considered in the computation of the feature vectors.

In [50]:
# Load data
true_links = {}
features = {}
validation_dict = {}
datasets= {'train':train_file, 
'test':test_file,
'validation':validate_file}

for key in datasets:
    df = pd.read_csv(datasets[key])
    nof_cols = int((df.shape[1] - 2)/2)
    dfA = df.iloc[:,2:nof_cols + 2] # left table
    dfB = df.iloc[:,nof_cols + 2:df.shape[1]] # right table
    # Delete 'left_' and 'right_' prefixes from column names
    dfA.rename(columns={c:c[5:] for c in dfA.columns },inplace=True) 
    dfB.rename(columns={c:c[6:] for c in dfB.columns },inplace=True)

    tuples = [(i,i) for i in range(len(df)) if df.iloc[i]['label'] == 1]
    true_links[key] = pd.MultiIndex.from_tuples(tuples)

    tuples_full = [(i,i) for i in range(len(df))]
    candidate_links = pd.MultiIndex.from_tuples(tuples_full)
    # Final features (used in all methods)
    features[key] = compare_cl.compute(candidate_links, dfA, dfB)

## PRLT Model Training
We are now ready to train the various models included in the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/). There are three super vised methods (Logistic regression, Naive Bayes and Support vector machines) and two unsupervised methods (K-Means and [ECM](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.18.3828&rep=rep1&type=pdf)). We also add a simple heuristic that we label as "Hand-tuned". This basically looks whether records match on the majority of columns (more ones than zeroes in the feature vector) and classifies a pair as a match if this is the case. We assume that this heuristic as a proxy for traditional programming methods as implemented in tools like Power BI, since these are also based on string edit distances. 

The results of the heuristic and the various models on the validation set is displayed below. As we can see, all the supervised  trained models clearly outperform the simple heuristic, whereas the best [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/) model only slightly underperforms the much more complicated and time-consuming [DeepMatcher](https://anhaidgroup.github.io/deepmatcher/html/) model. Interestingly, the unsupervised ECM model also is a significant gain compared to the simple heuristic, indicating that also without labels, a machine learning approach can be beneficial.

In [51]:
classifiers= {
'Hand-tuned':None,
'Logistic regression':recordlinkage.LogisticRegressionClassifier(),
'Naive Bayes': recordlinkage.NaiveBayesClassifier(),
'Support vector machine': recordlinkage.SVMClassifier(),
'K-means': recordlinkage.KMeansClassifier(),
'ECM': recordlinkage.ECMClassifier()}

for key in classifiers:
    validation_dict[key] = {}
    if key == 'Hand-tuned':
        # Immediate prediction
        result = features['validation'][features['validation'].sum(axis=1) > 2].index
    else:
        # Training 
        if key == 'ECM' or key == 'K-means':
            classifiers[key].fit(features['train']) 
        else:
            classifiers[key].fit(features['train'], true_links['train'])
        # Predict the match status for all test record pairs
        result =  classifiers[key].predict(features['validation'])
        
    # Validate
    validation_dict[key] = performance_metrics(true_links['validation'], result, len(features['validation']))
    
    #Print results
    print_results(key, validation_dict[key])

***********
Hand-tuned
***********
Confusion matrix:
[[  278   171]
 [   30 32100]]
Accuracy: 0.9938303815341171
Recall: 0.6191536748329621
F-score: 0.7344782034346102


********************
Logistic regression
********************
Confusion matrix:
[[  425    24]
 [   27 32103]]
Accuracy: 0.9984345744191043
Recall: 0.9465478841870824
F-score: 0.9433962264150944




c:\python-3.6.3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


************
Naive Bayes
************
Confusion matrix:
[[  434    15]
 [   28 32102]]
Accuracy: 0.9986801313729703
Recall: 0.9665924276169265
F-score: 0.9527991218441273


***********************
Support vector machine
***********************
Confusion matrix:
[[  425    24]
 [   27 32103]]
Accuracy: 0.9984345744191043
Recall: 0.9465478841870824
F-score: 0.9433962264150944


********
K-means
********
Confusion matrix:
[[  449     0]
 [ 3441 28689]]
Accuracy: 0.8943798152183923
Recall: 1.0
F-score: 0.20696012906199585


****
ECM
****
Confusion matrix:
[[  437    12]
 [   89 32041]]
Accuracy: 0.9968998434574419
Recall: 0.9732739420935412
F-score: 0.8964102564102564




## PRLT Testing
We select the best model based on the F1-measure on the vaildation set and give it the test set to classify. The result is displayed below. Notice that the result from DeepMatcher is only slightly better.

In [52]:
f_scores = {key:validation[key]['f-score'] for key in validation}
best_model = max(f_scores, key = f_scores.get) 
if best_model == 'Hand-tuned':
    result = features['test'][features['test'].sum(axis=1) > 2].index
else:
    result = classifiers[best_model].predict(features['test'])
test_dict =  performance_metrics(true_links['test'], result, len(features['test']))
print_results("Selected model ({}) on test set".format(best_model), test_dict)

*****************************************
Selected model (Naive Bayes) on test set
*****************************************
Confusion matrix:
[[  451    19]
 [   18 32091]]
Accuracy: 0.9988642990883698
Recall: 0.9595744680851064
F-score: 0.9605963791267307




## Conclusion
Our analysis seems to indicate that machine learning approaches can be beneficial in the context of entity matching, but that the question whether deep learning really outperforms more traditional M models can eb debated. Much will probably bedepend on the nature of the dataset and  the amount of time that one wished to invets in labeling data and training the model. An interesting observation is that at least on out not too complicated dataset, the unsupervised ECM method also show a very decent performance, which indicates that one can already gain soemthing from machine learning without labeling the data.

Furyher research could be geared towards the effect of fetaure engineering on the methods (DeepMatcher used other geatrues than PRLT) and on the question to what extent our simple heuristic was a good proxy to traditional programming methods implemented in existing entity matching systems.

In [57]:
dm_f1_test= model.run_eval(test)
dm_f1_validate = model.run_eval(validate)

===>  EVAL Epoch 5


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:11:35


Finished Epoch 5 || Run Time:  202.8 | Load Time:  494.7 || F1:  97.98 | Prec:  97.88 | Rec:  98.09 || Ex/s:  46.71

===>  EVAL Epoch 5


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:11:28


Finished Epoch 5 || Run Time:  200.4 | Load Time:  490.5 || F1:  98.43 | Prec:  98.88 | Rec:  98.00 || Ex/s:  47.16

